VGG Implement

In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils import data
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
writer = SummaryWriter(log_dir="./log")

In [3]:
class VGG_Module(nn.Module):
    def __init__(self, model_arch, in_dim=1):
        super().__init__()
        self.layer = []
        for (num, out_dim) in model_arch:
            self.layer.append(self.vgg_block(num, in_dim, out_dim))
            in_dim = out_dim
        self.model = nn.Sequential(*self.layer,
                                   nn.Flatten(),
                                   nn.Linear(out_dim * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
                                   nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
                                   nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
                                   nn.Linear(4096, 10))
    def vgg_block(self, num, in_dim, out_dim):
        layer = []

        for i in range(num):
            layer.append(nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1))
            layer.append(nn.ReLU())
            in_dim = out_dim
        layer.append(nn.MaxPool2d(kernel_size=2, stride=2))
        return nn.Sequential(*layer)
            
    def forward(self, x):
        return self.model(x)
    
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
model = VGG_Module(conv_arch)

print(model)

VGG_Module(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel

In [4]:
from tqdm import tqdm
    
def train(model, lr, epochs, train_loader, test_loader, writer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    global_step = 0
    test_step = 0
    optimer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        print(f"epoch: {epoch+1}/{epochs}")
        train_right = 0
        test_right = 0
        train_num = 0
        test_num = 0
        model.train()
        for X, y in tqdm(train_loader):
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            l = criterion(y_hat, y)
            optimer.zero_grad()
            l.backward()
            optimer.step()
            train_right += (y==y_hat.argmax(dim=1)).sum().item()
            train_num += len(y)

            writer.add_scalar("loss/train", l.item(), global_step)
            global_step += 1
            

        with torch.no_grad():
            model.eval()
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                l = criterion(y_hat, y)
                test_right += (y==y_hat.argmax(dim=1)).sum().item()
                test_num += len(y)

                writer.add_scalar("loss/test", l.item(), test_step)
                test_step += 1
        print(f"accuracy/train: {train_right/train_num}")
        print(f"aaccuracy/test: {test_right/test_num}")
        writer.add_scalar("accuracy/train", train_right/train_num, epoch)
        writer.add_scalar("accuracy/test", test_right/test_num, epoch)

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

data_dir = "./dataset"
trans = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])
train_data = torchvision.datasets.FashionMNIST(data_dir, train=True, transform=trans, download=False)
test_data = torchvision.datasets.FashionMNIST(data_dir, train=False, transform=trans, download=False)

lr = 0.001
batch_size = 256
train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

train(model, lr=lr, epochs=10, train_loader=train_loader, test_loader=test_loader, writer=writer)
writer.close()

epoch: 1/10


100%|██████████| 235/235 [02:14<00:00,  1.75it/s]


accuracy/train: 0.6038
aaccuracy/test: 0.8415
epoch: 2/10


100%|██████████| 235/235 [01:58<00:00,  1.98it/s]


accuracy/train: 0.8700833333333333
aaccuracy/test: 0.8735
epoch: 3/10


...

accuracy/train: 0.9392666666666667
aaccuracy/test: 0.9243
epoch: 9/10


100%|██████████| 235/235 [01:56<00:00,  2.01it/s]


accuracy/train: 0.9428833333333333
aaccuracy/test: 0.9278
epoch: 10/10


100%|██████████| 235/235 [01:54<00:00,  2.05it/s]


accuracy/train: 0.9466166666666667
aaccuracy/test: 0.9166


In [6]:
X, y = next(iter(train_loader))
print(X.shape)

torch.Size([256, 1, 224, 224])


In [ ]:
# torch.save(model.state_dict(), "./model.ckpt")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss()
with torch.no_grad():
    test_right = 0
    test_all = 0
    model.eval()
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        l = criterion(y_hat, y)
        test_right += (y==y_hat.argmax(dim=1)).sum().item()
        test_all += len(y)
    print(test_right/test_all)
        

0.9166
